# Cartographie avec d3

D3 contient un grand nombre de fonctions qui facilitent la création de cartes interactives. On peut également intégrer d3 avec Leaflet sans grand problème. Nous allons voir ici les bases de la cartographie thématique avec d3. Par contre, nous devons d'abord faire un petit détour par des aspects plus généraux d'optimisation de la cartographie interactive.

## 1. Faire mieux que du GeoJSON

Traditionnellement, des couches SIG vectorielles ont été stockées dans des fichiers Shape. Ce format n'étant pas très bien adapté à la cartographie sur le Web car il s'agit d'un format binaire qui en plus est réparti sur au moins 3 fichiers différents. Pour cette raison, le format GeoJson a vu le jour qui peut être considéré comme étant l'équivalent des fichiers Shape pour la cartographie sur le Web.

Un fichier GeoJSON a la structure suivante:

```json
{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": { "type": "Point", "coordinates": [36.706988, 55.664097] },
            "properties": { "name": "Khar Turan National Park" }
        },
        {
            "type": "Feature",
            "geometry": { "type": "Point", "coordinates": [37.547378, 56.381516] },
            "properties": { "name": "Golestan National Park" }
        }
    ]
}
```

Un fichier GeoJSON est facile à lire et à éditer, et avec QGIS ou des outils comme [ogr2ogr](http://www.gdal.org/ogr2ogr.html), il est facile d'en créer un à partir d'un fichier Shape p.ex.

Par contre, le problème principal des fichiers GeoJson est leur taille relativement importante même pour des régions relativement petites. C'est un problème lorsqu'on doit transférer ces fichiers par le Web. La taille importante des fichiers est dû à plusieurs raisons:

- Beaucoup d'éléments sont répétés, comme par exemple `"type": "Feature"`, `"geometry": { "type": "point", ... }`, ou encore les noms des attributs. L'exemple ci-dessus pourrait être écrit comme suit sans perdre de l'information:

        point                name
        36.706988,55.664097  Khar Turan National Park
        37.547378,56.381516  Golestan National Park
  
  ce qui fait 98 octets contre 459 octets, donc un _taux de compression_ de plus de 450%!
  

- La précision des coordonnées est souvent plus élevée que ce dont on a besoin pour la cartographie interactive. On n'a pas besoin d'avoir une précision au centimètre p.ex. On peut donc souvent enlever quelques décimales, style:

        point          name
        36.707,55.664  Khar Turan National Park
        37.547,56.382  Golestan National Park
  
  ce qui nous permet de réduire encore le volume de données


- Notamment pour les couches des polygones de limites administratives, très utilisées dans les cartes choroplèthes, quasiment chaque arrête de polygone est répétée. En effet, chaque unité administrative est définie par le polygone complet, et l'unité voisine répète la frontière commune. Ce format de données est appelé le __format spaghetti__, contrairement au __format topologique__ qui évite cette redondance. Le format topologique a par ailleurs l'avantage d'éviter des problèmes topologiques où la frontière commune n'est pas définie exactement de la même manière. Si nous changeons la façon de stocker nos géométries pour un format topologique, nous pouvons donc économiser presque la moitié des coordonnées...


- Beaucoup de couches vectorielles sont très peu généralisées et donc plus détaillées que nécessaire pour une cartographie thématique. En plus, le format spaghetti rend l'utilisation d'algorithmes automatiques de simplification difficile, en raison du risque d'introduire des erreurs topologiques. L'utilisation d'un format topologique facilite la simplification automatique. Ceci permet également d'adapter le niveau de généralisation au niveau de zoom plus facilement.


- Dans certaines situations, nous n'avons pas besoin de l'entier du fichier des géometries, p.ex. si on visualise uniquement une partie de la région lors d'un zoom. Il suffirait dans ce cas de transmettre uniquement une sous-région du fichier initial.


- Les fichiers JSON transmis contiennent du texte ASCII, et contient souvent uniquement environ 60 à 70 caractères différents, comparé aux 256 caractères possibles. Une traduction en format binaire optimisé pourrait donc optimiser presque deux tiers du volume de données transmis.

Pour palier à ces différents problèmes, il y a deux approches qui ont vu le jour ces dernières années:

1. le [format TopoJson](https://github.com/topojson/topojson) de Mike Bostock, un des principaux développeurs de d3
2. les [Vector Tiles](https://www.mapbox.com/vector-tiles/) de Mapbox

Le format TopoJson adresse tous les points mentionnés ci-dessus sauf les deux derniers, tandis que les Vector Tiles focalisent plus sur ces deux derniers points. Nous allons couvrir ici uniquement le format TopoJson.
